# Running Zhanghao Wu's code from his DeepCCA repo
link to the repo: https://github.com/Michaelvll/DeepCCA

If we want view2 to be SMILES, this paper could be helpful: https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-018-2523-5

In [1]:
import sys
sys.path.append('/home/cmdunham/mass_spec/mass_spec_repo/DeepCCA')

In [2]:
%load_ext autoreload
%autoreload 2
import torch
import torch.nn as nn
import numpy as np
from linear_cca import linear_cca
from torch.utils.data import BatchSampler, SequentialSampler
from DeepCCAModels import DeepCCA
from main import Solver
from utils import load_data, svm_classify
import random

import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
# try:
#     import cPickle as thepickle
# except ImportError:
#     import _pickle as thepickle

import gzip
import numpy as np
# torch.set_default_tensor_type(torch.DoubleTensor)

/home/cmdunham/mass_spec/mass_spec_env/lib/python3.8/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [7]:
spectra = pd.read_csv('/home/cmdunham/mass_spec/mass_spec_repo/data/scaled_spectra_with_instrument_type.csv')

In [ ]:
# create list of chemical names
spectra_names = [chem.split('.')[0] for chem in spectra.columns]

# separate spectrum data from chemical name encodings
spec_data = torch.tensor(spectra[:][:915].values, dtype=torch.float32).T 
encoded_names = torch.tensor(spectra[:][915:965].values, dtype=torch.float32).T

In [ ]:
# for spec in input_spectra:
#     noise = noise_pct * len(spec)
#     random_numbers = random.sample(range(len(spec)), int(round(noise)))
#     for i in random_numbers:
#         spec[i] = 0

In [ ]:
spec_vals = spectra.values.T

# Parameters for splitting
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Generate shuffled indices
num_samples = len(encoded_names)
indices = np.arange(num_samples)
np.random.shuffle(indices)

# Calculate split sizes
train_size = int(train_ratio * num_samples)
val_size = int(val_ratio * num_samples)
test_size = num_samples - train_size - val_size

# Split indices
train_indices = indices[:train_size]
val_indices = indices[train_size:train_size + val_size]
test_indices = indices[train_size + val_size:]

# Split data using indices
train_data1, val_data1, test_data1 = encoded_names[train_indices], encoded_names[val_indices], encoded_names[test_indices]
train_data2, val_data2, test_data2 = spec_vals[train_indices], spec_vals[val_indices], spec_vals[test_indices]

In [ ]:
data1 = torch.tensor(encoded_names, dtype=torch.float32)
data2 = torch.tensor(spectra.values, dtype=torch.float32).T
dataset = TensorDataset(data1, data2)
dataset = DataLoader(dataset=dataset, batch_size=32, shuffle=True)

In [5]:
############
# Parameters Section

device = torch.device('cuda')
print("Using", torch.cuda.device_count(), "GPUs")

# # the path to save the final learned features
# save_to = './new_features.gz'

# the size of the new space learned by the model (number of the new features)
outdim_size = 10

# size of the input for view 1 and view 2
input_shape1 = 50
input_shape2 = 977

# number of layers with nodes in each one
layer_sizes1 = [1024, 1024, 1024, outdim_size]
layer_sizes2 = [1024, 1024, 1024, outdim_size]

# the parameters for training the network
learning_rate = 1e-3
epoch_num = 10
batch_size = 32

# the regularization parameter of the network
# seems necessary to avoid the gradient exploding especially when non-saturating activations are used
reg_par = 1e-5

# specifies if all the singular values should get used to calculate the correlation or just the top outdim_size ones
# if one option does not work for a network or dataset, try the other one
use_all_singular_values = False

# if a linear CCA should get applied on the learned features extracted from the networks
# it does not affect the performance on noisy MNIST significantly
apply_linear_cca = True
# end of parameters section
############

# Each view is stored in a gzip file separately. They will get downloaded the first time the code gets executed.
# Datasets get stored under the datasets folder of user's Keras folder
# normally under [Home Folder]/.keras/datasets/
# data1 = load_data('./noisymnist_view1.gz')
# data2 = load_data('./noisymnist_view2.gz')

Using 2 GPUs


In [ ]:
data1[0][0]

tensor(1., dtype=torch.float32)

In [6]:
# Building, training, and producing the new features by DCCA
model = DeepCCA(layer_sizes1, layer_sizes2, input_shape1,
                input_shape2, outdim_size, use_all_singular_values, device=device).double()
l_cca = None
if apply_linear_cca:
    l_cca = linear_cca()
solver = Solver(model, l_cca, outdim_size, epoch_num, batch_size,
                learning_rate, reg_par, device=device)
train1, train2 = data1[0][0], data2[0][0]
val1, val2 = data1[1][0], data2[1][0]
test1, test2 = data1[2][0], data2[2][0]
# val1=None
# test1=None
solver.fit(train1, train2, val1, val2, test1, test2)
# TODO: Save linear_cca model if needed

set_size = [0, train1.size(0), train1.size(
    0) + val1.size(0), train1.size(0) + val1.size(0) + test1.size(0)]
loss, outputs = solver.test(torch.cat([train1, val1, test1], dim=0), torch.cat(
    [train2, val2, test2], dim=0), apply_linear_cca)

[ INFO : 2024-05-28 11:22:29,835 ] - DataParallel(
  (module): DeepCCA(
    (model1): MlpNet(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=50, out_features=1024, bias=True)
          (1): Sigmoid()
          (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
        (1-2): 2 x Sequential(
          (0): Linear(in_features=1024, out_features=1024, bias=True)
          (1): Sigmoid()
          (2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
        )
        (3): Sequential(
          (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
          (1): Linear(in_features=1024, out_features=10, bias=True)
        )
      )
    )
    (model2): MlpNet(
      (layers): ModuleList(
        (0): Sequential(
          (0): Linear(in_features=977, out_features=1024, bias=True)
          (1): Sigmoid()
          (2): BatchNorm1d(1024, e

NameError: name 'data1' is not defined

In [ ]:
new_data = []
# print(outputs)
for idx in range(3):
    new_data.append([outputs[0][set_size[idx]:set_size[idx + 1], :],
                     outputs[1][set_size[idx]:set_size[idx + 1], :], data1[idx][1]])
# Training and testing of SVM with linear kernel on the view 1 with new features
[test_acc, valid_acc] = svm_classify(new_data, C=0.01)
print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
print("Accuracy on view 1 (test data) is:", test_acc*100.0)

training SVM...
Accuracy on view 1 (validation data) is: 95.61
Accuracy on view 1 (test data) is: 95.1


In [ ]:
# Saving new features in a gzip pickled file specified by save_to
print('saving new features ...')
f1 = gzip.open(save_to, 'wb')
thepickle.dump(new_data, f1)
f1.close()

saving new features ...


In [ ]:
d = torch.load('checkpoint.model')
solver.model.load_state_dict(d)
solver.model.parameters()

<generator object Module.parameters at 0x7fc3f8183050>